In [2]:
import pandas as pd
import numpy as np

In [3]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [4]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer
)


load = Loader(
    filepath='benchmark://adult',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)
load.load()

/Users/alex/miniforge3/envs/dev310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loader - Benchmarker: file benchmark/adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.


In [5]:
import pprint


def issue332(
        load: Loader,
        synthesizing_method: str,
        scaler_inhibit: bool = False
):
    pp = pprint.PrettyPrinter(depth=2)

    proc_discretizing = Processor(
        metadata=load.metadata,
    )

    if scaler_inhibit:
        print("Preproc config of Scaler before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'scaler': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Scaler before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])

    proc_discretizing.fit(
        data=load.data,
        sequence=[
            'missing',
            'outlier',
            'scaler',
            'discretizing'
        ]
    )
    preproc_discretizing_data = proc_discretizing.transform(
        data=load.data
    )
    print("Preproc data as ...")
    print(preproc_discretizing_data.head(1))



    syn = Synthesizer(
        method=synthesizing_method,
        epsilon=10.0,
    )
    syn.create(data=preproc_discretizing_data)
    syn.fit_sample()
    print("Sync data as ...")
    print(syn.data_syn.head(1))

In [6]:
issue332(
    load=load,
    synthesizing_method='smartnoise-aim',
)

Preproc data as ...
   age  workclass  fnlwgt  education  educational-num  marital-status  \
0  0.0          3     0.0          1              0.0               4   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           6             3     2       1           0.0           0.0   

   hours-per-week  native-country  income  
0             0.0              38       0  
Synthesizer (SmartNoise): Fitting aim.


/Users/alex/miniforge3/envs/dev310/lib/python3.10/site-packages/mbi/__init__.py:15: UserWarning: MixtureInference disabled, please install jax and jaxlib
  warnings.warn('MixtureInference disabled, please install jax and jaxlib')


ValueError: The transformer appears to have some continuous columns. Please provide only categorical or ordinal.